# Merton Model Asset Value Analysis

This notebook provides comprehensive analysis of the asset values estimated by the Merton model.

## Analysis Components:
1. **Overall Statistics** - Distribution of asset values and volatilities
2. **Firm-Level Analysis** - Per-firm summaries and characteristics
3. **Leverage Analysis** - Debt-to-equity ratios and their implications
4. **Temporal Patterns** - How asset values evolve over time
5. **Volatility Analysis** - Distribution and patterns in asset volatility estimates

In [ ]:
# Setup
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils import config

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✓ Setup complete')

## 1. Load Data

In [ ]:
# Load Merton estimation results
df = pd.read_csv(config.OUTPUT_DIR / 'merged_data_with_merton.csv')
df['date'] = pd.to_datetime(df['date'])

# Load daily returns
daily_returns = pd.read_csv(config.OUTPUT_DIR / 'daily_asset_returns.csv')
daily_returns['date'] = pd.to_datetime(daily_returns['date'])

print(f'Loaded {len(df):,} observations')
print(f'Date range: {df["date"].min()} to {df["date"].max()}')
print(f'Number of firms: {df["gvkey"].nunique()}')
print(f'\nColumns: {df.columns.tolist()}')

## 2. Overall Statistics

In [ ]:
print('='*80)
print('ASSET VALUE STATISTICS')
print('='*80)
print(df['asset_value'].describe())
print(f'\nMissing values: {df["asset_value"].isna().sum():,} ({df["asset_value"].isna().mean()*100:.1f}%)')

print('\n' + '='*80)
print('ASSET VOLATILITY STATISTICS (Annualized)')
print('='*80)
# Convert from daily to annualized for display
df['asset_volatility_annual'] = df['asset_volatility'] * np.sqrt(252)
print(df['asset_volatility_annual'].describe())
print(f'\nMissing values: {df["asset_volatility"].isna().sum():,} ({df["asset_volatility"].isna().mean()*100:.1f}%)')

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Asset value distribution
axes[0, 0].hist(df['asset_value'] / 1e9, bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Asset Value (Billions €)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Asset Values')

# Asset value distribution (log scale)
axes[0, 1].hist(np.log10(df['asset_value']), bins=50, edgecolor='black')
axes[0, 1].set_xlabel('Log10(Asset Value)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Asset Values (Log Scale)')

# Volatility distribution
axes[1, 0].hist(df['asset_volatility_annual'] * 100, bins=50, edgecolor='black')
axes[1, 0].set_xlabel('Asset Volatility (% p.a.)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of Asset Volatility (Annualized)')
axes[1, 0].axvline(df['asset_volatility_annual'].median() * 100, color='red', linestyle='--', label='Median')
axes[1, 0].legend()

# Market cap vs Asset value
axes[1, 1].scatter(df['mkt_cap'] / 1e9, df['asset_value'] / 1e9, alpha=0.1, s=1)
axes[1, 1].set_xlabel('Market Cap (Billions €)')
axes[1, 1].set_ylabel('Asset Value (Billions €)')
axes[1, 1].set_title('Market Cap vs Asset Value')
axes[1, 1].plot([0, df['mkt_cap'].max()/1e9], [0, df['mkt_cap'].max()/1e9], 'r--', alpha=0.5, label='45° line')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'merton_asset_value_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print('✓ Saved: merton_asset_value_distributions.png')

## 3. Firm-Level Analysis

In [ ]:
# Calculate statistics per firm
firm_stats = df.groupby(['gvkey', 'company']).agg({
    'asset_value': ['mean', 'std', 'min', 'max', 'count'],
    'asset_volatility_annual': ['mean', 'std'],
    'mkt_cap': 'mean',
    'liabilities_total': 'mean'
}).reset_index()

# Flatten column names
firm_stats.columns = ['gvkey', 'company', 'asset_value_mean', 'asset_value_std', 
                      'asset_value_min', 'asset_value_max', 'n_obs',
                      'volatility_mean', 'volatility_std',
                      'mkt_cap_mean', 'liabilities_mean']

# Calculate leverage
firm_stats['leverage'] = firm_stats['liabilities_mean'] / firm_stats['mkt_cap_mean']

# Convert to billions for readability
for col in ['asset_value_mean', 'asset_value_std', 'asset_value_min', 'asset_value_max', 
            'mkt_cap_mean', 'liabilities_mean']:
    firm_stats[f'{col}_bn'] = firm_stats[col] / 1e9

# Sort by mean asset value
firm_stats_sorted = firm_stats.sort_values('asset_value_mean', ascending=False)

print('\nFIRM-LEVEL SUMMARY (Top 10 by mean asset value):')
print(firm_stats_sorted[[
    'company', 'asset_value_mean_bn', 'volatility_mean', 'leverage', 'n_obs'
]].head(10).to_string(index=False))

# Export full table
firm_stats_sorted.to_csv(config.OUTPUT_DIR / 'merton_firm_level_statistics.csv', index=False)
print('\n✓ Saved: merton_firm_level_statistics.csv')

In [ ]:
# Visualize firm-level characteristics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Top 15 firms by mean asset value
top_firms = firm_stats_sorted.head(15)
axes[0, 0].barh(range(len(top_firms)), top_firms['asset_value_mean_bn'])
axes[0, 0].set_yticks(range(len(top_firms)))
axes[0, 0].set_yticklabels(top_firms['company'], fontsize=8)
axes[0, 0].set_xlabel('Mean Asset Value (Billions €)')
axes[0, 0].set_title('Top 15 Firms by Mean Asset Value')
axes[0, 0].invert_yaxis()

# Volatility by firm
axes[0, 1].barh(range(len(top_firms)), top_firms['volatility_mean'] * 100)
axes[0, 1].set_yticks(range(len(top_firms)))
axes[0, 1].set_yticklabels(top_firms['company'], fontsize=8)
axes[0, 1].set_xlabel('Mean Asset Volatility (% p.a.)')
axes[0, 1].set_title('Top 15 Firms: Asset Volatility')
axes[0, 1].invert_yaxis()

# Leverage by firm
leverage_sorted = firm_stats.sort_values('leverage', ascending=False).head(15)
axes[1, 0].barh(range(len(leverage_sorted)), leverage_sorted['leverage'])
axes[1, 0].set_yticks(range(len(leverage_sorted)))
axes[1, 0].set_yticklabels(leverage_sorted['company'], fontsize=8)
axes[1, 0].set_xlabel('Leverage (Debt/Equity)')
axes[1, 0].set_title('Top 15 Firms by Leverage Ratio')
axes[1, 0].invert_yaxis()
axes[1, 0].axvline(20, color='red', linestyle='--', alpha=0.5, label='Leverage = 20')
axes[1, 0].legend()

# Asset value stability (coefficient of variation)
firm_stats['cv'] = firm_stats['asset_value_std'] / firm_stats['asset_value_mean']
cv_sorted = firm_stats.sort_values('cv', ascending=False).head(15)
axes[1, 1].barh(range(len(cv_sorted)), cv_sorted['cv'])
axes[1, 1].set_yticks(range(len(cv_sorted)))
axes[1, 1].set_yticklabels(cv_sorted['company'], fontsize=8)
axes[1, 1].set_xlabel('Coefficient of Variation (std/mean)')
axes[1, 1].set_title('Top 15 Firms by Asset Value Variability')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'merton_firm_level_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print('✓ Saved: merton_firm_level_analysis.png')

## 4. Leverage Analysis

Examining the relationship between leverage and asset value estimates.

In [ ]:
# Calculate leverage for all observations
df['leverage'] = df['liabilities_total'] / df['mkt_cap']

print('='*80)
print('LEVERAGE ANALYSIS')
print('='*80)
print(df['leverage'].describe())
print(f'\nFirms with leverage > 20: {firm_stats[firm_stats["leverage"] > 20]["company"].tolist()}')
print(f'\nFirms with leverage > 30: {firm_stats[firm_stats["leverage"] > 30]["company"].tolist()}')

In [ ]:
# Leverage vs Volatility
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(df['leverage'], df['asset_volatility_annual'] * 100, alpha=0.1, s=1)
axes[0].set_xlabel('Leverage (Debt/Equity)')
axes[0].set_ylabel('Asset Volatility (% p.a.)')
axes[0].set_title('Leverage vs Asset Volatility')
axes[0].set_xlim(0, 100)
axes[0].axvline(20, color='red', linestyle='--', alpha=0.5, label='Leverage = 20 (high)')
axes[0].legend()

# Binned analysis
leverage_bins = [0, 5, 10, 20, 50, 150]
df['leverage_bin'] = pd.cut(df['leverage'], bins=leverage_bins, labels=['0-5', '5-10', '10-20', '20-50', '50+'])
leverage_volatility = df.groupby('leverage_bin')['asset_volatility_annual'].agg(['mean', 'median', 'std', 'count'])

x_pos = range(len(leverage_volatility))
axes[1].bar(x_pos, leverage_volatility['mean'] * 100, yerr=leverage_volatility['std'] * 100, 
            capsize=5, alpha=0.7, edgecolor='black')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(leverage_volatility.index)
axes[1].set_xlabel('Leverage Group')
axes[1].set_ylabel('Mean Asset Volatility (% p.a.)')
axes[1].set_title('Asset Volatility by Leverage Group')
axes[1].grid(axis='y', alpha=0.3)

# Add counts
for i, (idx, row) in enumerate(leverage_volatility.iterrows()):
    axes[1].text(i, row['mean'] * 100 + row['std'] * 100 + 5, 
                f'n={int(row["count"]):,}', ha='center', fontsize=8)

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'merton_leverage_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print('✓ Saved: merton_leverage_analysis.png')

## 5. Temporal Analysis

Examining how asset values and volatilities evolve over time.

In [ ]:
# Aggregate by date
temporal_stats = df.groupby('date').agg({
    'asset_value': ['mean', 'median', 'std'],
    'asset_volatility_annual': ['mean', 'median'],
    'leverage': ['mean', 'median']
}).reset_index()

temporal_stats.columns = ['date', 'asset_value_mean', 'asset_value_median', 'asset_value_std',
                          'volatility_mean', 'volatility_median', 
                          'leverage_mean', 'leverage_median']

# Plot
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Asset values over time
axes[0].plot(temporal_stats['date'], temporal_stats['asset_value_mean'] / 1e9, label='Mean')
axes[0].plot(temporal_stats['date'], temporal_stats['asset_value_median'] / 1e9, label='Median')
axes[0].fill_between(temporal_stats['date'], 
                     (temporal_stats['asset_value_mean'] - temporal_stats['asset_value_std']) / 1e9,
                     (temporal_stats['asset_value_mean'] + temporal_stats['asset_value_std']) / 1e9,
                     alpha=0.3)
axes[0].set_ylabel('Asset Value (Billions €)')
axes[0].set_title('Cross-Sectional Mean Asset Value Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volatility over time
axes[1].plot(temporal_stats['date'], temporal_stats['volatility_mean'] * 100, label='Mean')
axes[1].plot(temporal_stats['date'], temporal_stats['volatility_median'] * 100, label='Median')
axes[1].set_ylabel('Asset Volatility (% p.a.)')
axes[1].set_title('Cross-Sectional Mean Asset Volatility Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].axhline(20, color='red', linestyle='--', alpha=0.3, label='20% vol')

# Leverage over time
axes[2].plot(temporal_stats['date'], temporal_stats['leverage_mean'], label='Mean')
axes[2].plot(temporal_stats['date'], temporal_stats['leverage_median'], label='Median')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Leverage (Debt/Equity)')
axes[2].set_title('Cross-Sectional Mean Leverage Over Time')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].axhline(20, color='red', linestyle='--', alpha=0.3, label='Leverage = 20')

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'merton_temporal_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print('✓ Saved: merton_temporal_analysis.png')

## 6. Asset Return Analysis

Examining the daily asset returns calculated from Merton model.

In [ ]:
print('='*80)
print('ASSET RETURN STATISTICS')
print('='*80)
print(daily_returns['asset_return_daily'].describe())
print(f'\nExtreme returns (|return| > 50%): {(np.abs(daily_returns["asset_return_daily"]) > 0.5).sum():,}')
print(f'Extreme returns (|return| > 100%): {(np.abs(daily_returns["asset_return_daily"]) > 1.0).sum():,}')

In [ ]:
# Return distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Full distribution
returns_winsorized = daily_returns['asset_return_daily'].clip(-0.5, 0.5)  # For visualization
axes[0].hist(returns_winsorized * 100, bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Daily Asset Return (%)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Daily Asset Returns (winsorized at ±50% for display)')
axes[0].axvline(0, color='red', linestyle='--', alpha=0.5)

# Q-Q plot
from scipy import stats
returns_clean = daily_returns['asset_return_daily'].dropna()
returns_clean = returns_clean[(np.abs(returns_clean) < 0.5)]  # Remove extreme outliers for Q-Q plot
stats.probplot(returns_clean, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot: Asset Returns vs Normal Distribution')

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'merton_asset_return_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('✓ Saved: merton_asset_return_distribution.png')

## Summary

This analysis provides comprehensive insights into the Merton model asset value estimations:

- **Asset values** range widely across firms, reflecting differences in company size and leverage
- **Volatility** varies significantly, with higher volatility often associated with higher leverage
- **Leverage** is particularly high for financial institutions (banks, insurance), which can cause numerical instability
- **Temporal patterns** reflect macroeconomic cycles and crisis periods (2011-2012 European debt crisis, 2020 COVID)

**Key Findings:**
- High-leverage firms (leverage > 20×) show more volatile asset value estimates
- Financial institutions require special handling due to their extreme leverage ratios
- Asset volatility estimates are generally reasonable (10-50% p.a.) for most firms